# DACT : Dataset Automatic Crawling Tools

### 검색엔진에서 입력한 키워드의 이미지를 찾아 자동으로 임시 폴더(temp)에 다운로드
### temp의 이미지파일을 7:3 비율로 Train Set, Test Set으로 분할하여 검색어 이름의 폴더로 분할한다.

#### v1.1 : PATH 안정화
#### v1.2 : tqdm을 이용한 진행도 표시
#### v2.0 : 키워드 리스트 저장후 멀티 크롤링 가능
#### v2.1 : 코드셀 병합, log print 규격화
#### v2.2 : 모든 항목 총 소요시간 계산 후 출력
#### v2.3 : test, train 경로 버그 수정
#### v3.0 : keras 최신 이미지 전처리 기술과 동기화하여 새로운 디렉토리 생성
#### v4.0 : deleter 1.0과 병합하여 크롤링과 데이터 검열 동시에 수행, 데이터셋 폴더 지정가능, 접근성 강화

#### 세원아토스 기술연구소 조영탁

In [15]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

import json
import os
import argparse
import sys


import requests
import urllib
import urllib3
from urllib3.exceptions import InsecureRequestWarning

import datetime
import time
import glob
import numpy as np
from PIL import Image

urllib3.disable_warnings(InsecureRequestWarning)

#searclist에 키워드 입력 (다중 입력 가능)
#searchlist = 'tops for ladies', 'ladies outerwear', 'dress', 'ladies underwear', 'women suit', 'women pants'
#searchlist = '여자 니트', 'tops for ladies',  '여자 맨투맨', '여자 민소매', '여자 블라우스', '여자 셔츠', '여자 티셔츠', 'lady t-shirt', '여자 상의'
#searchlist = '여자 아우터', 'ladies outerwear',  '여자 가디건', '여자 자켓', '여자 점퍼', '여자 집업', '여자 조끼 베스트', '여자 코트', '여자 외투'
#searchlist = '여자 원피스 -수영복', 'dress',  '여자 롱 원피스', '여자 미니 원피스', '여성 원피스', '오버롤 원피스', '여자 드레스', '여성 원피스 패션', '원피스 신상'
#searchlist = 'women pants', '여성 바지',  '여성 슬랙스', '여성 청바지', '여성 코튼팬츠', '여자 코튼팬츠', '여자 슬랙스', '여성 신상 팬츠', '여성 와이드팬츠'
#searchlist = 'lady skirts', 'women skirt',  '롱 스커트', '숏 스커트', '스커트 신상', '여성 스커트', '여성 치마', '여자 스커트', '치마 신상'
#searchlist = '여자 투피스 수트', '여성 수트',  'women suit', '여자 투피스 정장', '여성 정장', '여자 정장', '여자 블레이저', '여성 블레이저', 'lady formal suit'

############INSERT##############
searchlist = 'dog', 'cat' #검색어 리스트
pdirs = 'animal' #class 이름 입력
path = 'testdataset' # 다운로드 경로 입력
############INSERT##############


namecount = 1000
searchcount = len(searchlist)
temp_dirs = 'temp'

maxcount = 500 # 700개 이상 하위 노출 데이터는 질이 급감함.
os.chdir('C:\Projects\keras_talk')

chromedriver = 'C://Program Files//chromedriver//chromedriver.exe'
#Import Chromedriver -  절대경로를 지정

def download_google_staticimages(searchword):
    
    
    if not os.path.exists(path +'/'):
        os.mkdir(path +'/')
    
    if not os.path.exists(path +'/' + pdirs):
        os.mkdir(path +'/'+ pdirs)
        
            
    searchurl = 'https://www.google.com/search?q=' + searchword + '&source=lnms&tbm=isch'
    options = webdriver.ChromeOptions()
    options.add_argument('--no-sandbox')
    #options.add_argument('--headless')

    try:
        browser = webdriver.Chrome(chromedriver, options=options)
    except Exception as e:
        print(f'No found chromedriver in this environment.')
        print(f'Install on your machine. exception: {e}')
        sys.exit()

    browser.set_window_size(1280, 1024)
    browser.get(searchurl)
    time.sleep(1)

    element = browser.find_element_by_tag_name('body')

    # Scroll down
    #for i in range(30):
    for i in range(50):
        element.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.3)

    try:
        browser.find_element_by_id('smb').click()
        for i in range(50):
            element.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.3)
    except:
        for i in range(10):
            element.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.3)

    time.sleep(0.5)
    time.sleep(0.5)

    browser.find_element_by_xpath('//input[@value="결과 더보기"]').click() #

    # Scroll down 2
    for i in range(50):
        element.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.3)

    try:
        browser.find_element_by_id('smb').click()
        for i in range(50):
            element.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.3)
    except:
        for i in range(10):
            element.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.3)
            
    print('     - Scrolling -> OK')
    page_source = browser.page_source 

    soup = BeautifulSoup(page_source, 'lxml')
    images = soup.find_all('img')

    urls = []
    for image in images:
        try:
            url = image['data-src']
            if not url.find('https://'):
                urls.append(url)
        except:
            try:
                url = image['src']
                if not url.find('https://'):
                    urls.append(image['src'])
            except Exception as e:
                print(f'     - No found image sources.')
                print(e)

    count = 0
    if urls:
        for url in tqdm(urls):
            try:
                res = requests.get(url, verify=False, stream=True)
                rawdata = res.raw.read()
                with open(os.path.join(path +'/'+ pdirs + '/', searchword + str(count) + '.jpg'), 'wb') as f:
                    f.write(rawdata)
                    count += 1
            except Exception as e:
                print('     - Failed to write rawdata.')
                print(e)      
                
                
    os.path.join('C:\Projects\keras_talk')
    browser.close()
    return count
    


# Main block
def main():
    t0 = time.time()
    for i in range(searchcount):
        print('::: ' + searchlist[i] + ' -> Start Crawling. (' + str(i+1) + '/' + str(len(searchlist)) +')' + ' :::')
        count = download_google_staticimages(searchlist[i])
        print(f'\n')
    t1 = time.time()
    total_time = t1 - t0
    print(f'\n')
    print('::: '+str(len(searchlist)) + f'개 항목 총 소요 시간 : {str(total_time)} seconds.' + ' :::')
    print(f'\n')
    
    count = 0
    os.chdir('C:/Projects/keras_talk/'+ path+ '/' + pdirs)
    img_list = glob.glob('*.*')

    for i in range(len(img_list)):
        im = Image.open(img_list[i])

        if((im.size[0] * 3.3 < im.size[1]) or (im.size[0] > im.size[1] * 3.3)) :
            im.close()
            #os.remove(img_list[i])
            print('remove : ' + img_list[i])
            count = count + 1

    print(str(count) + '개 항목 삭제 완료')

if __name__ == '__main__':
    main()

remove : cat287.jpg
remove : cat89.jpg
remove : dog637.jpg
3개 항목 삭제 완료
